# 1. 层和块

① nn.Sequential 定义了一种特殊的Module。

In [2]:
# 回顾一下多层感知机
import torch
from torch import nn  #多层感知机用的
from torch.nn import functional as F  #新引入的
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10)) 
X = torch.rand(2,20) # ! 此处的X定义为：批量大小 × 输入特征维度
net(X)

tensor([[ 0.2927,  0.3841, -0.2454, -0.1671, -0.0177,  0.0539, -0.0561, -0.0582,
         -0.2100,  0.0497],
        [ 0.1875,  0.3142, -0.2425, -0.0179,  0.0388,  0.0378, -0.0926,  0.0218,
         -0.1183,  0.1043]], grad_fn=<AddmmBackward0>)

# 2. 自定义块

* 任何的层或者神经网络都可以认为是Module的子类
* 自定义块的两个关键函数：__init__ 和 forward

In [3]:
import torch
from torch import nn  #多层感知机用的
from torch.nn import functional as F  #新引入的
class MLP(nn.Module): # 定义为nn.Module的子类
    def __init__(self): 
        super().__init__()  # 调用父类的__init__函数
        self.hidden = nn.Linear(20,256) # 全连接层1：隐藏层
        self.out = nn.Linear(256,10) # 全连接层2：输出层
        
    # 定义前向函数，self指向自己，类似this指针
    # 解决给定输入怎么做输出的问题
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
# 实例化多层感知机的层，然后在每次调用正向传播函数调用这些层
net = MLP()
X = torch.rand(2,20)
net(X)

tensor([[ 0.2096, -0.0666, -0.2179,  0.2932,  0.0938, -0.1016, -0.2259,  0.1050,
         -0.1918,  0.1939],
        [ 0.1813,  0.0934, -0.0984,  0.1598,  0.1004,  0.0786, -0.1924, -0.0483,
         -0.1504,  0.2789]], grad_fn=<AddmmBackward0>)

# 3. 顺序块

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        # 接收了一个list of input arguments，用于存储liner的信息
        super().__init__()
        for block in args:
            self._modules[block] = block # block 本身作为它的key，存在_modules里面的为层，以字典的形式 
            
    def forward(self, X):
        for block in self._modules.values():
            print(block)
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[-0.1893, -0.0072,  0.0360, -0.0730, -0.1440, -0.1782,  0.3141,  0.1610,
         -0.0613, -0.0355],
        [-0.0546, -0.0718,  0.0739, -0.2900, -0.1168, -0.1704,  0.2054,  0.1324,
         -0.0412, -0.1227]], grad_fn=<AddmmBackward0>)

# 4. 正向传播

In [5]:
# 通过继承 nn.Module ，更加灵活的定义网络与前向计算
# 不使用Sequential，自定义一个类，可以进行大量的自定义的计算
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False) # 随机生成权重，但是不计算梯度，不参与训练
        self.linear = nn.Linear(20,20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight + 1))  # 矩阵乘法加1
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()  # 返回一个标量
    
net = FixedHiddenMLP()
X = torch.rand(2,20)
net(X)

tensor(-0.3970, grad_fn=<SumBackward0>)

# 5. 混合组合块（嵌套）

In [ ]:
# 理解：嵌套使用，因为现有的层和网络都是nn.Module的子类
# 混合代培各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
X = torch.rand(2,20)
chimera(X)

tensor(0.0644, grad_fn=<SumBackward0>)

# 6. 参数管理

In [7]:
# 首先关注具有单隐藏层的多层感知机
import torch
from torch import nn
 
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))  # Sequential可以理解成一个list
X = torch.rand(size=(2,4))
print(net(X))

tensor([[0.1100],
        [0.1520]], grad_fn=<AddmmBackward0>)


In [9]:
print(net[2].state_dict()) # 访问参数，net[2]就是最后一个输出层，state就是该层的参数，对线形层就是权重和偏移

OrderedDict([('weight', tensor([[-0.2282,  0.1345,  0.0975,  0.3067,  0.1830, -0.2462, -0.0100,  0.0749]])), ('bias', tensor([0.1546]))])


In [11]:
print(type(net[2].bias)) # 目标参数
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1546], requires_grad=True)
tensor([0.1546])


In [13]:
print(net[2].weight.grad == None) # 还没进行反向计算，所以grad为None

True


In [ ]:
# 一次性访问一层中的所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])  
# 一次性访问整个网络的所有参数。0是第一层名字，1是ReLU，它没有参数         
print(*[(name, param.shape) for name, param in net.named_parameters()])  

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [ ]:
print(net.state_dict()['2.bias'].data) # 通过名字获取参数

# 7. 嵌套块

In [ ]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

# block2可以嵌套四个block1 
def block2():
    net = nn.Sequential()
    for i in range(4):
        # ! 使用add_module()可以传入一个字符串作为块的名字，而不是自动命名为0，1，2，3……
        net.add_module(f'block{i}',block1())                                      
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
print(rgnet(X))
# 第0层为一个Sequential嵌套块，嵌套块中含有4个Sequential子块（4层），每个Sequential子块有4层
print(rgnet)

tensor([[-0.2778],
        [-0.2778]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# 8 内置初始化

In [16]:
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))

def init_normal(m):
    if type(m) == nn.Linear:
        # ! 下划线写法表示替换函数，即把m.weight的值替换掉，而不会返回值   
        nn.init.normal_(m.weight, mean=0, std=0.01) 
        nn.init.zeros_(m.bias)
        
net.apply(init_normal) # 会递归调用 直到net里的所有层都初始化
print(net[0].weight.data[0])
print(net[0].bias.data[0])

tensor([ 0.0169, -0.0060, -0.0078,  0.0119])
tensor(0.)


In [ ]:
# 不建议将参数初始化为固定常数
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
print(net[0].weight.data[0]) 
print(net[0].bias.data[0])

tensor([1., 1., 1., 1.])
tensor(0.)


In [ ]:
# 对不同的层应用不同的初始化
def xavier(m):
    """
    Xavier 初始化旨在解决上述梯度消失/爆炸问题，它试图让各层激活值的方差保持一致
    具体采用均匀分布来生成初始权重。它根据当前层的输入特征数量（fan_in）和输出特征数量（fan_out）来计算一个合适的取值范围 [ -bound, bound ]
    bound= sqrt(6 / (fan_in+fan_out) )
    """
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) # 宇宙的答案
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.3779, -0.3074, -0.2969, -0.5504],
        [ 0.0260, -0.1049, -0.6233, -0.2050],
        [-0.0937, -0.2387, -0.4923,  0.4747],
        [ 0.0792,  0.5286,  0.2346,  0.0479],
        [-0.0352,  0.1823, -0.1764, -0.4183],
        [-0.2254,  0.2669,  0.6601, -0.0258],
        [-0.2206,  0.3543,  0.6917,  0.5300],
        [-0.6918,  0.3496,  0.4863, -0.2998]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [ ]:
# ! 'ReLU' object has no attribute 'weight'
def wrong(m):
    nn.init.constant_(m.weight, 42)
net.apply(wrong)

AttributeError: 'ReLU' object has no attribute 'weight'

In [19]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",*[(name, param.shape) for name, param in m.named_parameters()][0])  # 打印名字是啥，形状是啥       
        nn.init.uniform_(m.weight, -10, 10) # 均匀初始化
        m.weight.data *= m.weight.data.abs() >=  5 # 这里*=的代码相当于先计算一个布尔矩阵(先判断>=)，然后再用布尔矩阵的对应元素去乘以原始矩阵的每个元素。保留绝对值大于5的权重，不是的话就设为0

net.apply(my_init)
print(net[0].weight[:2])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[-0., -0., -0., 0.],
        [-0., -0., -0., 0.]], grad_fn=<SliceBackward0>)


# 9. 参数替换

In [20]:

net[0].weight.data[:] += 1 # 参数替换
net[0].weight.data[0,0] = 42
print(net[0].weight.data[0])

tensor([42.,  1.,  1.,  1.])


# 10. 参数绑定

In [21]:
# 在一些层之前共享权重
# 参数绑定
shared = nn.Linear(8,8) # 1.先构造要共享的层
# 第2个隐藏层和第3个隐藏层是share权重的，第一个和第四个是自己的  
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8,1))  
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data )
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([[ 1.0000e+02,  3.3404e-01, -6.1526e-02,  1.9173e-01, -2.0091e-01,
         -3.1250e-01, -2.8254e-01,  1.3433e-01],
        [ 2.8021e-01, -2.5047e-01,  3.1191e-01, -1.1795e-01, -1.4305e-01,
          1.5253e-01,  1.3559e-01, -3.1633e-02],
        [-3.0541e-01, -3.2833e-02, -3.9448e-02, -7.9433e-02,  2.7043e-01,
         -3.2510e-01,  1.5304e-01,  3.0713e-01],
        [-5.4374e-02, -2.2438e-01, -6.7887e-02,  3.6033e-02,  7.6232e-02,
         -2.8566e-01, -3.5316e-01, -1.7697e-01],
        [ 1.3097e-01,  2.0349e-01, -2.5389e-01,  7.4717e-02, -3.0244e-01,
         -1.9146e-01,  3.4112e-01, -1.2492e-01],
        [-3.1494e-01, -2.6830e-01, -3.4181e-01, -1.9149e-01, -8.7546e-02,
         -1.6958e-01, -1.7600e-01,  3.1310e-01],
        [ 6.9221e-02,  2.2611e-01,  2.9718e-01, -2.5945e-01, -1.8921e-01,
          1.2128e-01,  2.1989e-01, -1.0804e-01],
        [ 2.0490e-01,  8.9103e-02,  1.8829e-01, -3.4979e-01, -1.2753e-01,
        

# 11. 自定义层

In [22]:
# 与自定义网络本质相同，同样是nn.Module的子类
# 构造一个没有任何参数的自定义层
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()   # 输入减均值，使得均值变成0
    
layer = CenteredLayer()
print(layer(torch.FloatTensor([1,2,3,4,5]))) # 输入数据经过该层计算后输出数据的均值为0


tensor([-2., -1.,  0.,  1.,  2.])


### 将层作为组件合并到构建更复杂的模型中

In [ ]:

net = nn.Sequential(nn.Linear(8,128),CenteredLayer())
Y = net(torch.rand(4,8))
Y.mean()

In [25]:
# 带参数的层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # ! 参与训练的参数需要放入Parameter类中
        # nn.Parameter使得这些参数加上了梯度  输入大小乘以输出大小
        self.weight = nn.Parameter(torch.randn(in_units, units)) 
        self.bias = nn.Parameter(torch.randn(units, ))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5,3)
dense.weight


Parameter containing:
tensor([[ 1.2892, -0.2561, -0.3563],
        [-0.3121,  2.2503, -0.4068],
        [-0.4203, -1.3587,  1.2919],
        [ 0.8984,  0.4531, -0.6226],
        [-1.3111,  2.1612,  0.6617]], requires_grad=True)

In [ ]:
# dense是一个实例化的MyLinear层
# 使用自定义层直接执行正向传播计算
print(dense(torch.rand(2,5))) # 批量大小×特征维数

# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64,8), MyLinear(8,1))
print(net(torch.rand(2,64)))

tensor([[0.0000, 0.4227, 1.7563],
        [0.6538, 0.2539, 1.4771]])
tensor([[12.8311],
        [ 9.1199]])


# 12. 读写文件

### 加载和保存张量（怎么存一个矩阵）

In [ ]:

import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(5)
torch.save(x, './save/x-file1') # 将工程中的张量保存到文件中
x2 = torch.load('./save/x-file1') # 将文件中的张量加载到工程中
print(x2)

tensor([0, 1, 2, 3, 4])


### 存储一个张量列表，然后把它们读回内存

In [29]:

y = torch.zeros(4)
torch.save([x,y],'./save/x-files')
x2, y2 = torch.load('./save/x-files')
print(x2)
print(y2)

tensor([0, 1, 2, 3, 4])
tensor([0., 0., 0., 0.])


### 写入或读取从字符串映射到张量的字典

In [30]:

mydict = {'x':x,'y':y}
torch.save(mydict,'./save/mydict')
mydict2 = torch.load('./save/mydict')
print(mydict2)

{'x': tensor([0, 1, 2, 3, 4]), 'y': tensor([0., 0., 0., 0.])}


### 加载和保存模型参数

In [32]:
# 网络中有什么可以存的参数 - 权重
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

# 将模型的参数存储为一个叫做"mlp.params"的文件
torch.save(net.state_dict(),'./save/mlp.params')

# 实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数
clone = MLP() # 必须要先声明一下，才能使用load_state_dict()导入参数
clone.load_state_dict(torch.load("./save/mlp.params"))
print(clone.eval()) # eval()是进入测试模式

Y_clone = clone(X)
print(Y_clone == Y)

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)
tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
